In [1]:
from datasets import load_dataset
from transformers import pipeline

In [2]:
dataset = load_dataset("heegyu/news-category-dataset", split="train")

Using custom data configuration heegyu--news-category-dataset-a0dcb53f17af71bf


Extracting data files: 100%|██████████| 1/1 [00:00<00:00, 499.68it/s]
                                

Dataset json downloaded and prepared to C:/Users/heegyukim/.cache/huggingface/datasets/heegyu___json/heegyu--news-category-dataset-a0dcb53f17af71bf/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab. Subsequent calls will reuse this data.


In [3]:
import torch
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
pipe = pipeline("text-generation", "gpt2", device=device)
pipe.model.config.pad_token_id = pipe.tokenizer.eos_token_id

In [4]:
dataset[0]

{'link': 'https://www.huffpost.com/entry/covid-boosters-uptake-us_n_632d719ee4b087fae6feaac9',
 'headline': 'Over 4 Million Americans Roll Up Sleeves For Omicron-Targeted COVID Boosters',
 'category': 'U.S. NEWS',
 'short_description': 'Health experts said it is too early to predict whether demand would match up with the 171 million doses of the new boosters the U.S. ordered for the fall.',
 'authors': 'Carla K. Johnson, AP',
 'date': datetime.datetime(2022, 9, 23, 0, 0)}

In [5]:
from pprint import pprint


def generate_news(item, num_return_sequences=1):
  headline = item['headline']
  desc = item['short_description']
  prompt = f"{headline}\n{desc}\n"
  texts = pipe(
      prompt, 
      max_new_tokens=128, 
      num_return_sequences=num_return_sequences,
      do_sample=True
      )
  prompt_len = len(prompt)

  texts = [x['generated_text'][prompt_len:].strip() for x in texts]
  return texts

pprint(generate_news(dataset[0]))

['The bulk of these doses are given when young adults and children use COVID, '
 'such as for the COVA, to boost their blood sugar and heart rate.\n'
 '\n'
 'About 3.8 million kids have heard omicron-targeted booster doses of COVID. '
 'Researchers say the average cost for a child who uses omicron-targeted COVID '
 'at home is $100.\n'
 '\n'
 'CoVID can be injected directly into the eyes or other large portions of the '
 'body, or with injections to the chest or the upper back. The administration '
 'process involved administering each dose to a child at home.\n'
 '\n'
 'Experts say the doses']


In [7]:
import jsonlines
from tqdm.notebook import tqdm

max_items = 20000

with jsonlines.open("data/gpt-huffpost.jsonl", "w") as f:
  for i, item in tqdm(enumerate(dataset), total=max_items):
    if i == max_items:
      break
      
    preds = generate_news(item, 1)
    item['predictions'] = preds
    del item['date']
    f.write(item)

  0%|          | 0/20000 [00:00<?, ?it/s]

d:\Anaconda3\lib\site-packages\transformers\pipelines\base.py:1043: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
